In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [49]:
import pandas as pd
import numpy as np
import random as rn
import tensorflow as tf
from tensorflow import keras
import joblib
import math

# keras random seed https://junstar92.tistory.com/98

In [85]:
train = pd.read_csv('train_pm_shift.csv', index_col=0)
train.drop(['PM2.5_24'], axis=1, inplace=True)

type0 = train[train['측정소']  == 0].iloc[-1, 0]
type1 = train[train['측정소']  == 1].iloc[-1, 0]
type2 = train[train['측정소']  == 2].iloc[-1, 0]
type3 = train[train['측정소']  == 3].iloc[-1, 0]
type4 = train[train['측정소']  == 4].iloc[-1, 0]
type5 = train[train['측정소']  == 5].iloc[-1, 0]
type6 = train[train['측정소']  == 6].iloc[-1, 0]
type7 = train[train['측정소']  == 7].iloc[-1, 0]
type8 = train[train['측정소']  == 8].iloc[-1, 0]
type9 = train[train['측정소']  == 9].iloc[-1, 0]
type10 = train[train['측정소']  == 10].iloc[-1, 0]
type11 = train[train['측정소']  == 11].iloc[-1, 0]
type12 = train[train['측정소']  == 12].iloc[-1, 0]
type13 = train[train['측정소']  == 13].iloc[-1, 0]
type14 = train[train['측정소']  == 14].iloc[-1, 0]
type15 = train[train['측정소']  == 15].iloc[-1, 0]
type16 = train[train['측정소']  == 16].iloc[-1, 0]

train = train.drop(train[train['월'] == 12.0].index, axis=0)
train = train.reset_index(drop=True)

train = pd.get_dummies(train, columns=['월', '시', '일'])
#train.drop(['측정소'], axis=1, inplace=True)
train

,PM2.5,기온,풍향,풍속,강수량,습도,측정소,PM2.5_1,2017,2018,...,일_22.0,일_23.0,일_24.0,일_25.0,일_26.0,일_27.0,일_28.0,일_29.0,일_30.0,일_31.0
0,0.044,0.173952,0.192577,0.031119,0.000000,0.769354,11,0.000,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0.048,0.172526,0.231582,0.034610,0.000000,0.766737,11,0.044,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0.060,0.169968,0.197688,0.027797,0.000000,0.762224,11,0.048,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0.064,0.165598,0.087934,0.022631,0.000000,0.757474,11,0.060,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0.064,0.155845,0.198305,0.020634,0.000000,0.781484,11,0.064,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545491,0.240,0.475922,0.182945,0.017162,0.000009,0.825206,8,0.236,0,0,...,0,0,0,0,0,0,0,0,1,0
545492,0.212,0.474407,0.301395,0.019875,0.000051,0.829761,8,0.240,0,0,...,0,0,0,0,0,0,0,0,1,0
545493,0.236,0.468954,0.164688,0.018752,0.000009,0.840570,8,0.212,0,0,...,0,0,0,0,0,0,0,0,1,0
545494,0.216,0.463321,0.130414,0.018005,0.000000,0.860967,8,0.236,0,0,...,0,0,0,0,0,0,0,0,1,0


In [87]:
test_y = pd.read_csv('test_y.csv', index_col=0)
test = pd.read_csv('test_concat.csv', index_col=0)
test.drop(['PM2.5', 'PM2.5_24'], axis=1, inplace=True)
test = pd.concat([test_y, test], axis=1)

test.iloc[7728*0, 10] = type0
test.iloc[7728*1, 10] = type1
test.iloc[7728*2, 10] = type2
test.iloc[7728*3, 10] = type3
test.iloc[7728*4, 10] = type4
test.iloc[7728*5, 10] = type5
test.iloc[7728*6, 10] = type6
test.iloc[7728*7, 10] = type7
test.iloc[7728*8, 10] = type8
test.iloc[7728*9, 10] = type9
test.iloc[7728*10, 10] = type10
test.iloc[7728*11, 10] = type11
test.iloc[7728*12, 10] = type12
test.iloc[7728*13, 10] = type13
test.iloc[7728*14, 10] = type14
test.iloc[7728*15, 10] = type15
test.iloc[7728*16, 10] = type16

test = pd.get_dummies(test, columns=['월', '시', '일'])
#test.drop(['측정소'], axis=1, inplace=True)
test

,PM2.5,기온,풍향,풍속,강수량,습도,측정소,PM2.5_1,2017,2018,...,일_22.0,일_23.0,일_24.0,일_25.0,일_26.0,일_27.0,일_28.0,일_29.0,일_30.0,일_31.0
0,0.060,0.243999,0.206394,0.039352,0.000000,0.669021,0,0.064,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.064,0.230344,0.218569,0.029155,0.000000,0.693413,0,0.060,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.072,0.217311,0.153686,0.020029,0.000000,0.731035,0,0.064,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.064,0.209426,0.264642,0.019601,0.000000,0.754623,0,0.072,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.056,0.202601,0.108036,0.018393,0.000000,0.781270,0,0.064,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131371,0.056,0.422899,0.806253,0.115426,0.000011,0.524092,16,0.068,0,0,...,0,0,0,0,0,0,0,0,0,0
131372,0.036,0.413532,0.779885,0.112505,0.000011,0.553151,16,0.056,0,0,...,0,0,0,0,0,0,0,0,0,0
131373,0.020,0.405441,0.778800,0.115319,0.000021,0.586888,16,0.036,0,0,...,0,0,0,0,0,0,0,0,0,0
131374,0.024,0.396523,0.758119,0.122272,0.000000,0.604689,16,0.020,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# train = pd.read_csv('train.csv', index_col=0)
# test_y = pd.read_csv('/content/test_y.csv', index_col=0)
# test = pd.read_csv('test.csv', index_col=0)
# test = pd.concat([test_y, test], axis=1)
# train

In [88]:
df = train

timestep_input = 48
timestep_output = 72
timestep = timestep_input + timestep_output


Encoder_X_lst = []
encoder_x = []
for i in range(17):
    df = train[train['측정소'] == i]
    for i in range( len(df)//(timestep_input+timestep_output)): 
        encoder_x.extend([df.iloc[i*timestep : i*timestep + timestep_input, :].values])
Encoder_X = np.array(encoder_x)
print(Encoder_X.shape)



Decoder_X_lst = []
decoder_x = []
for i in range(17):
    df = train[train['측정소'] == i]
    for i in range( len(df)//(timestep_input+timestep_output) ):
        decoder_x.extend([df.iloc[i*timestep + timestep_input : i*timestep + timestep_input + timestep_output, 6:].values])
Decoder_X = np.array(decoder_x).astype('float')
print(Decoder_X.shape)



Decoder_Y_lst = []
decoder_y = []
for i in range(17):
    df = train[train['측정소'] == i]
    for i in range( len(df)//(timestep_input+timestep_output) ):
        decoder_y.extend([df.iloc[i*timestep + timestep_input : i*timestep + timestep_input + timestep_output, 0].values.T])
Decoder_Y = np.array(decoder_y)
print(Decoder_Y.shape)

# Encoder_X_lst = []
# encoder_x = []
# for i in range( 0, len(df) - timestep_output - timestep_input): 
#     encoder_x.extend([df.iloc[i : i + timestep_input, :].values])
# Encoder_X = np.array(encoder_x)
# print(Encoder_X.shape)


# Decoder_X_lst = []
# decoder_x = []
# for i in range( 0, len(df) - timestep_output - timestep_input ):
#     decoder_x.extend([df.iloc[i + timestep_input : i + timestep_input + timestep_output, 6:].values])  
# Decoder_X = np.array(decoder_x)
# print(Decoder_X.shape)


# Decoder_Y_lst = []
# decoder_y = []
# for i in range( 0, len(df) - timestep_output - timestep_input ):
#     decoder_y.extend([df.iloc[i + timestep_input : i + timestep_input + timestep_output, 0].values.T])
# Decoder_Y = np.array(decoder_y)
# print(Decoder_Y.shape)

(4539, 48, 79)
(4539, 72, 73)
(4539, 72)


# 모델링

In [89]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
rn.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

from keras import optimizers, initializers, losses

EPOCHS = 200
LEARNING_RATE = 0.0001
OPTIMIZER = optimizers.Adam(learning_rate=LEARNING_RATE)
INITIALIZER = initializers.HeNormal(seed=RANDOM_SEED)
LOSS = losses.MeanSquaredError()

In [90]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self,
                 batch_size: int,
                 shuffle: bool = False) -> None:
        
        self.encoder_x = Encoder_X
        self.decoder_x = Decoder_X
        self.y = Decoder_Y

        self.batch_size = batch_size
        self.shuffle = shuffle

        self.on_epoch_end()

    def on_epoch_end(self):
        self.index = np.arange(len(self.encoder_x))
        if self.shuffle:
            np.random.shuffle(self.index)


    def __len__(self):
        return math.ceil(len(self.encoder_x) / self.batch_size)

    def __getitem__(self, index):
        batch_index = self.index[index*self.batch_size:(index+1)*self.batch_size]

        encoder_x_batch = self.encoder_x[ [i for i in batch_index] ]

        decoder_x_batch = self.decoder_x[ [i for i in batch_index] ]

        y_batch = self.y[ [i for i in batch_index] ]

        return [encoder_x_batch,decoder_x_batch], y_batch

In [91]:
train_gen = DataGenerator(batch_size = 128, shuffle=True)

In [92]:
# Early Stopping

from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='loss',         
                   min_delta=0.01,                
                   patience=15,                 
                   verbose=1,
                   restore_best_weights=True)  

In [93]:
# 8.43

tf.random.set_seed(RANDOM_SEED)
tf.keras.backend.clear_session()

from keras.models import Model
from keras.layers import Input, LSTM, Dense, TimeDistributed

BATCH_SIZE = None

ENCODER_INPUT_DIM = train_gen.encoder_x.shape[2]  # 11
DECODER_INPUT_DIM = train_gen.decoder_x.shape[2]  # 11

ENCODER_TIMESTEP = 48
DECODER_TIMESTEP = 72
LATENT_DIM = 256

# rnn 계열은 무조건 tanh 사용해야 gpu학습 가능.

# Encoder

encoder_inputs = Input(shape=(ENCODER_TIMESTEP, ENCODER_INPUT_DIM))

lstm_enc_0, latent_c_0, latent_h_0 = LSTM(LATENT_DIM, activation='tanh', return_sequences=True, return_state=True)(encoder_inputs)

lstm_enc, latent_c_1, latent_h_1 = LSTM(LATENT_DIM, activation='tanh', return_sequences=True, return_state=True)(lstm_enc_0)

_, latent_c_2, latent_h_2 = LSTM(LATENT_DIM, activation='tanh', return_sequences=False, return_state=True)(lstm_enc) # return_sequences = True는 timestep별 출력 , return_state = True, last_hidden_state 출력

lstm0_output = [latent_c_0, latent_h_0]
lstm1_output = [latent_c_1, latent_h_1]
lstm2_output = [latent_c_2, latent_h_2]

# Decoder

decoder_inputs = Input(shape=(DECODER_TIMESTEP, DECODER_INPUT_DIM))

lstm_dec_0 = LSTM(LATENT_DIM, activation='tanh', return_sequences=True)(decoder_inputs, initial_state=lstm0_output)

lstm_dec = LSTM(LATENT_DIM, activation='tanh', return_sequences=True)(lstm_dec_0, initial_state=lstm1_output)

step, latent_c, latent_h = LSTM(LATENT_DIM, activation='tanh', return_sequences=True, return_state=True)(lstm_dec,initial_state=lstm2_output)


# Regressor

# TimeDistributed
# https://medium.com/smileinnovation/how-to-work-with-time-distributed-data-in-a-neural-network-b8b39aa4ce00

# regressor_outputs = TimeDistributed(Dense(512, activation='relu'))(step)

# regressor_outputs = Dense(512, activation='relu')(step) # 1,759,233
regressor_result = TimeDistributed(layer = Dense(1))(step) 

   
model = Model([encoder_inputs, decoder_inputs], regressor_result)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 48, 79)]     0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 72, 73)]     0           []                               
                                                                                                  
 lstm (LSTM)                    [(None, 48, 256),    344064      ['input_1[0][0]']                
                                 (None, 256),                                                     
                                 (None, 256)]                                                     
                                                                                              

In [94]:
model.compile(loss=LOSS, optimizer=OPTIMIZER)
model.fit(train_gen, epochs=100, callbacks=[es], shuffle=True)

Epoch 1/100
36/36 [==============================] - 10s 61ms/step - loss: 0.0098
Epoch 2/100
36/36 [==============================] - 2s 59ms/step - loss: 0.0042
Epoch 3/100
36/36 [==============================] - 2s 59ms/step - loss: 0.0038
Epoch 4/100
36/36 [==============================] - 2s 59ms/step - loss: 0.0036
Epoch 5/100
36/36 [==============================] - 2s 59ms/step - loss: 0.0034
Epoch 6/100
36/36 [==============================] - 2s 60ms/step - loss: 0.0031
Epoch 7/100
36/36 [==============================] - 2s 59ms/step - loss: 0.0030
Epoch 8/100
36/36 [==============================] - 2s 60ms/step - loss: 0.0026
Epoch 9/100
36/36 [==============================] - 2s 60ms/step - loss: 0.0024
Epoch 10/100
36/36 [==============================] - 2s 60ms/step - loss: 0.0023
Epoch 11/100
36/36 [==============================] - 2s 60ms/step - loss: 0.0023
Epoch 12/100
36/36 [==============================] - 2s 60ms/step - loss: 0.0021
Epoch 13/100
36/36 [====

In [95]:
# import joblib


model.save('/content/drive/MyDrive/pm/model_lstm_1.h5')

In [96]:
test['측정소'] = test['측정소'].astype(int)


Encoder_X_lst = []
encoder_x = []

for i in range(17):

    df = test[test['측정소'] == i]

    for i in range( len(df)//(timestep_input+timestep_output)): 
        encoder_x.extend([df.iloc[i*timestep : i*timestep + timestep_input, :].values])

Encoder_X = np.array(encoder_x)
print(Encoder_X.shape)



Decoder_X_lst = []
decoder_x = []

for i in range(17):

    df = test[test['측정소'] == i]

    for i in range( len(df)//(timestep_input+timestep_output) ):
        decoder_x.extend([df.iloc[i*timestep + timestep_input : i*timestep + timestep_input + timestep_output, 6:].values])
        
Decoder_X = np.array(decoder_x).astype('float')
print(Decoder_X.shape)


Decoder_Y_lst = []
decoder_y = []

for i in range(17):

    df = test[test['측정소'] == i]

    for i in range( len(df)//(timestep_input+timestep_output) ):
        decoder_y.extend([df.iloc[i*timestep + timestep_input : i*timestep + timestep_input + timestep_output, 0].values.T])
        
Decoder_Y = np.array(decoder_y)
print(Decoder_Y.shape)

(1088, 48, 79)
(1088, 72, 73)
(1088, 72)


In [97]:
test_encoder_x = Encoder_X
test_decoder_x = Decoder_X

In [98]:
y_pred = model.predict([test_encoder_x, test_decoder_x])

34/34 [==============================] - 2s 13ms/step


In [99]:
result = pd.read_csv('/content/drive/MyDrive/pm/answer_sample.csv')
result['PM2.5'] = y_pred.reshape(y_pred.shape[0]*y_pred.shape[1], 1)

In [102]:
result.to_csv('lstm_v5.csv', index=False, encoding='utf-8')

In [103]:
result.isnull().sum()

연도       0
일시       0
측정소      0
PM2.5    0
dtype: int64